# Basic NN

This IJulia shows the basic concept and application with neural network, which has some matrix calculus.

Firstly we set $$X = [x^{(1)}, x^{(2)}, ..., x^{(m)}]$$ as the inputted data, where $x^{i}$ is an $n$ by $1$ vector with $n$ features. Then we have an $N_{i-1}$ by $N_{i}$ vector $w^{[i]}$ to represent the _weight matrix_ at the $i$th layer and an $N_i$ by $1$ bias vector $b^{[i]}$.  

Thus at each layer we could have the calculation:
$$z^{[i]} = w^{[i]\mathrm{T}}a^{[i-1]} + b^{[i]}\\a^{[i]} = \sigma(z^{[i]})$$
where $a^{[i]}$ is the activation ouput from layer $i$. 

It is all known that there are a lot of vector and matrix derivatives in NN. So we need to analyze them completely in order to get a deep comprehenson of NN.

We construct a fully connected NN here: 
<center><img src="pic/342NN.png" width=80%/></center>
with 4 layers which have N, 3, 4 and 2 nodes in each layer. This may look like a binary classification applied in tasks such as picture classification(However CNN performs much better in these problems).

## Cost/Loss Function

To train a NN, we need some rules and directions to point out that how to adjust our $w$s and $b$s. And that is __Cost Funcion__ and __Loss Function__.<br/>
Loss function is used to measure the performance of NN describing the "distance" bewteen the output of our NN and the true labels from dataset. Usually we would like to minimize the loss function to make NN work better. In the former NN we constructed, we have two output values. And we need a loss function $$L: \mathcal{R}^2 \rightarrow \mathcal{R}$$ Here $L$ must be a functional, or it would generate a couple of conflicting NNs.

Because in each layer, the weight edges are independent to each other, it's same to calculate $\frac{\partial L}{\partial w^{[i]}}$, $\frac{\partial L}{\partial w^{[i]}_j}$ and $\frac{\partial L}{\partial w^{[i]}_{jk}}$ for updating the weights. For convience, we will using the matrix derivative to calculate.

## Gradients

With loss function, what we need to do in training NN is using the gradient to update weights and bias
$$
w^{[i]} := w^{[i]} - \alpha\nabla_{w^{[i]}}L \\
b^{[i]} := b^{[i]} - \alpha\nabla_{b^{[i]}}L
$$
and with BGD, we could have
$$
w^{[i]} := w^{[i]} - \alpha\frac{1}{m}\sum_{j=1}^m\nabla_{w^{[i]}}L^{(j)} \\
b^{[i]} := b^{[i]} - \alpha\frac{1}{m}\sum_{j=1}^m\nabla_{b^{[i]}}L^{(j)}
$$
So all we need to do is to calculate $\nabla_{w^{[i]}}L$ and $\nabla_{b^{[i]}}L$. With [chain rule](https://en.wikipedia.org/wiki/Chain_rule) applied, we have
$$\begin{align*}
\left(\nabla_{w^{[3]}_j}L\right)^\mathrm{T}
    &= \frac{\partial L}{\partial w^{[3]}_j} \\
    &= \left(\frac{\partial L}{\partial a^{[3]}}\right)_{1\times2}
       \left(\frac{\partial a^{[3]}}{\partial z^{[3]}}\right)_{2\times2}
       \left(\frac{\partial z^{[3]}}{\partial w^{[3]}_j}\right)_{2\times4} \\
\end{align*}$$
where $w^{[3]}_j = w^{[3]}_{:,j}$. And because of independency, we could have
$$\begin{align*}
\left(\nabla_{w^{[3]}}L\right)^\mathrm{T}
    &= \frac{\partial L}{\partial w^{[3]}} \\
    &= \sum_{j=1}^2(E_j)_{2\times1}\left(\frac{\partial L}{\partial a^{[3]}}\right)_{1\times2}
       \left(\frac{\partial a^{[3]}}{\partial z^{[3]}}\right)_{2\times2}
       \left(\frac{\partial z^{[3]}}{\partial w^{[3]}_j}\right)_{2\times4} \\
\end{align*}$$
where $E_1 = [1,0]^\mathrm{T}, E_2 = [0,1]^\mathrm{T}$. Samely, we could easily calculate $\nabla_{b^{[3]}}L$.

To get a more suitable form, we need to inspect $\nabla_{w^{[2]}}L$,
$$\begin{align*}
\left(\nabla_{w^{[2]}_j}L\right)^\mathrm{T}
    &= \frac{\partial L}{\partial w^{[2]}_j} \\
    &= \left(\frac{\partial L}{\partial a^{[3]}}\right)_{1\times2}
       \left(\frac{\partial a^{[3]}}{\partial z^{[3]}}\right)_{2\times2}
       \left(\frac{\partial z^{[3]}}{\partial a^{[2]}}\right)_{2\times4}
       \left(\frac{\partial a^{[2]}}{\partial z^{[2]}}\right)_{4\times4}
       \left(\frac{\partial z^{[2]}}{\partial w^{[2]}_j}\right)_{4\times3}
\end{align*}$$
and 
$$
\left(\nabla_{w^{[2]}}L\right)^\mathrm{T}
    =  \sum_{j=1}^4\left(E_j\right)_{4\times1}\left(\frac{\partial L}{\partial a^{[3]}}\right)_{1\times2}
       \left(\frac{\partial a^{[3]}}{\partial z^{[3]}}\right)_{2\times2}
       \left(\frac{\partial z^{[3]}}{\partial a^{[2]}}\right)_{2\times4}
       \left(\frac{\partial a^{[2]}}{\partial z^{[2]}}\right)_{4\times4}
       \left(\frac{\partial z^{[2]}}{\partial w^{[2]}_j}\right)_{4\times3}
$$

## Vectorization & Broadcasting

We have known that the built in functions for matrix calculations perform much better than explicit loop operation in Julia, as well as Python. So we'd like to find a more matrix tasty calculation, where we need the [Vectorization](https://en.wikipedia.org/wiki/Vectorization_%28mathematics%29).

Because of the independency between $w_j$ in the same layer, instead of calculating $\nabla_{w}L$, we can calculate
$$\begin{align*}
\left(\nabla_{\mathrm{vec}(w^{[3]})}L\right)^\mathrm{T}
    &= \frac{\partial L}{\partial \mathrm{vec}({w^{[3]}})} \\
    &= \left(\frac{\partial L}{\partial a^{[3]}}\right)_{1\times2}
       \left(\frac{\partial a^{[3]}}{\partial z^{[3]}}\right)_{2\times2}
       \left(\frac{\partial z^{[3]}}{\partial \mathrm{vec}(w^{[3]})}\right)_{2\times8} \\
    &= \left[\frac{\partial L}{\partial {w^{[3]}_1}}, \frac{\partial L}{\partial {w^{[3]}_2}}\right]_{1\times8} \\
    &= \left(\mathrm{vec}(\nabla_{w^{[3]}}L)\right)^\mathrm{T}
\end{align*}$$
And with _reshape_ function, we could transform it easily.

Now let's look at the most complicated gradient
$$\begin{align*}
\left(\nabla_{\mathrm{vec}(w^{[1]})}L\right)^\mathrm{T}
    &= \left(\frac{\partial L}{\partial a^{[3]}}\right)_{1\times2}
       \left(\frac{\partial a^{[3]}}{\partial z^{[3]}}\right)_{2\times2}
       \left(\frac{\partial z^{[3]}}{\partial a^{[2]}}\right)_{2\times4}
       \left(\frac{\partial a^{[2]}}{\partial z^{[2]}}\right)_{4\times4}
       \left(\frac{\partial z^{[2]}}{\partial a^{[1]}}\right)_{4\times3}
       \left(\frac{\partial a^{[1]}}{\partial z^{[1]}}\right)_{3\times3}
       \left(\frac{\partial z^{[1]}}{\partial \mathrm{vec}(w^{[1]})}\right)_{3\times3N} \\
    &= \left(\nabla L(a^{[3]\mathrm{T}})\right)
       \left(diag\left(\nabla\sigma(z^{[3]}_j)\right)\right)
       w^{[3]\mathrm{T}}
       \left(diag\left(\nabla\sigma(z^{[2]}_j)\right)\right)
       w^{[2]\mathrm{T}}
       \left(diag\left(\nabla\sigma(z^{[1]}_j)\right)\right)
       \left(diag\left(x^{(i)\mathrm{T}},x^{(i)\mathrm{T}},x^{(i)\mathrm{T}}\right)\right)
\end{align*}$$

Here $\nabla L,\nabla\sigma$ in the right are all map functions.

And because the diagonal matrix(and block matrix for $diag(x^{(i)\mathrm{T}}, x^{(i)\mathrm{T}}, x^{(i)\mathrm{T}})$) has many zeros, we'd better use __broadcasting__ to save the space. So we have
$$\begin{align*}
\left(\nabla_{w^{[1]}}L\right)_{N\times3}
    &=  \left(\nabla L(a^{[3]\mathrm{T}})\right)_{1\times2} .*
       \left(\nabla\sigma(z^{[3]\mathrm{T}}_j)\right)_{1\times2} *
       \left(w^{[3]\mathrm{T}}\right)_{2\times4} .*
       \left(\nabla\sigma(z^{[2]\mathrm{T}}_j)\right)_{1\times4} *
       \left(w^{[2]\mathrm{T}}\right)_{4\times3} .*
       \left(\nabla\sigma(z^{[1]\mathrm{T}}_j)\right)_{1\times3} .*
       \left[x^{(i)},x^{(i)},x^{(i)}\right]_{N\times 3} \\
    &=  \left(\nabla L(a^{[3]\mathrm{T}})\right)_{1\times2} .*
       \left(\nabla\sigma(z^{[3]\mathrm{T}}_j)\right)_{1\times2} *
       \left(w^{[3]\mathrm{T}}\right)_{2\times4} .*
       \left(\nabla\sigma(z^{[2]\mathrm{T}}_j)\right)_{1\times4} *
       \left(w^{[2]\mathrm{T}}\right)_{4\times3} .*
       \left(\nabla\sigma(z^{[1]\mathrm{T}}_j)\right)_{1\times3} .*
       \left[x^{(i)}\right]_{N\times 1}
\end{align*}$$
where $.*$ is broadcasting in Julia and $*$ is the normal matrix multiplication. Thanks to broadcasting, we don't need to reshape the matrix. One example for broadcasting is:

In [7]:
a = [1 2 3]  # 1 by 3
b = [1:10;]  # 10 by 1
println(a .* b, size(a .* b))  # 10 by 3
println(a .* a, size(a .* a))  # 1 by 3

[1 2 3; 2 4 6; 3 6 9; 4 8 12; 5 10 15; 6 12 18; 7 14 21; 8 16 24; 9 18 27; 10 20 30](10, 3)
[1 4 9](1, 3)


This is same for $b^{[1]}$:
$$\begin{align*}
\left(\nabla_{b^{[1]}}L\right)^\mathrm{T}
    &=  \left(\nabla L(a^{[3]\mathrm{T}})\right)_{1\times2} .*
       \left(\nabla\sigma(z^{[3]\mathrm{T}}_j)\right)_{1\times2} *
       \left(w^{[3]\mathrm{T}}\right)_{2\times4} .*
       \left(\nabla\sigma(z^{[2]\mathrm{T}}_j)\right)_{1\times4} *
       \left(w^{[2]\mathrm{T}}\right)_{4\times3} .*
       \left(\nabla\sigma(z^{[1]\mathrm{T}}_j)\right)_{1\times3} * E_{3\times3} \\
    &= \left(\nabla L(a^{[3]\mathrm{T}})\right)_{1\times2} .*
       \left(\nabla\sigma(z^{[3]\mathrm{T}}_j)\right)_{1\times2} *
       \left(w^{[3]\mathrm{T}}\right)_{2\times4} .*
       \left(\nabla\sigma(z^{[2]\mathrm{T}}_j)\right)_{1\times4} *
       \left(w^{[2]\mathrm{T}}\right)_{4\times3} .*
       \left(\nabla\sigma(z^{[1]\mathrm{T}}_j)\right)_{1\times3}
\end{align*}$$

Thus we could have:
$$\begin{align*}
\delta^{[i]} 
    &=  \left(\nabla L(a^{[D]\mathrm{T}})\right)_{1\times N_D} \\
    &.* \left(\nabla\sigma(z^{[D]\mathrm{T}}_j)\right)_{1\times N_D} * 
        \left(w^{[D]\mathrm{T}}\right)_{N_D\times N_{D-1}} \\
    &.*  \left(\nabla\sigma(z^{[D-1]\mathrm{T}}_j)\right)_{1\times N_{D-1}} *
       \left(w^{[D-1]\mathrm{T}}\right)_{N_{D-1}\times N_{D-1}} \\
    &.* \cdots \\
    &.*  \left(\nabla\sigma(z^{[i+1]\mathrm{T}}_j)\right)_{1\times N_{i+1}} *
       \left(w^{[i+1]\mathrm{T}}\right)_{N_{i+1}\times N_{i}} \\
    &.* \left(\nabla\sigma(z^{[i]\mathrm{T}}_j)\right)_{1\times N_{i}}
\end{align*}$$
and
$$
\nabla_{w^{[i]}}L = \delta^{[i]} .* a^{[i-1]} \\
\nabla_{b^{[i]}}L = \delta^{[i]\mathrm{T}}
$$
also to save time and space, we would have
$$
\delta^{[i]} = \delta^{[i+1]} 
    * \left(w^{[i+1]\mathrm{T}}\right)_{N_{i+1}\times N_{i}}
    .* \left(\nabla\sigma(z^{[i]\mathrm{T}}_j)\right)_{1\times N_{i}}
$$
So every time we get $z^{[i]}$, we could send it to calculate $w^{[i+1]\mathrm{T}}.* \nabla\sigma(z^{[i]\mathrm{T}}_j)$ and store it for the backpropagation. Note that 
$$A_{1\times N}*B_{N\times M}.*C_{1\times M} = A*(B.*C)$$
According to BGD, we need to calculate different $z^{[i](j)}$ with input $x^{(j)}$, and could we simplify it in matrix form rather than put then in a for-loop? 

Let's think about $\nabla_{w^{[1]}}L$ with $M$ samples.
$$\begin{align*}
\sum_{i=1}^M\left(\nabla_{w^{[1]}}L^{(i)}\right)_{N\times3}
    &=  \sum_{i=1}^M\left\{\left(\nabla L(a^{[3]\mathrm{T}}_M)\right)_{M\times2} .*
       \left(\nabla\sigma(z^{[3]\mathrm{T}}_M)\right)_{M\times2} *
       \left(w^{[3]\mathrm{T}}\right)_{2\times4} .*
       \left(\nabla\sigma(z^{[2]\mathrm{T}}_M)\right)_{M\times4} *
       \left(w^{[2]\mathrm{T}}\right)_{4\times3} .*
       \left(\nabla\sigma(z^{[1]\mathrm{T}}_M)\right)_{M\times3}\right\}_i .*
       \left[x^{(i)}\right]_{N\times 1} \\
    &= X_{N\times M}
       \left[\left(\nabla L(a^{[3]\mathrm{T}}_M\mathrm{T})\right)_{M\times2} .*
       \left(\nabla\sigma(z^{[3]\mathrm{T}}_M)\right)_{M\times2} *
       \left(w^{[3]\mathrm{T}}\right)_{2\times4} .*
       \left(\nabla\sigma(z^{[2]\mathrm{T}}_M)\right)_{M\times4} *
       \left(w^{[2]\mathrm{T}}\right)_{4\times3} .*
       \left(\nabla\sigma(z^{[1]\mathrm{T}}_M)\right)_{M\times3}\right]\\
    &\triangleq X\delta^{[1]}_M
\end{align*}$$
in the same way
$$
\sum_{i=1}^M\nabla_b^{[1]}L^{(i)} = \delta^{[1]\mathrm{T}}_M\mathbf{1}_v
$$
and
$$
\delta^{[i]}_M = \delta^{[i+1]}_M
    * \left(w^{[i+1]\mathrm{T}}\right)_{N_{i+1}\times N_{i}}
    .* \left(\nabla\sigma(z^{[i]})\right)_{M\times N_{i}}
$$
Thanks to broadcasting, it only needs a little difference to apply on matrix in BGD.
However, in the matrix form, we could not have $A*B.*C = A*(B.*C)$, for $B_{P\times Q}.*C_{M\times Q}$ is not defined in broadcasting. So we could only calculate it in its origin order.

## Example
Let's see an example.